In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms

In [2]:
# Build up the class for Vanilla FFN
class VanillaFFN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(VanillaFFN, self).__init__()
        self.fc1 = nn.Linear(in_features=input_size, out_features=input_size // 2)
        self.fc2 = nn.Linear(in_features=input_size // 2, out_features=num_classes)
    
    def forward(self, x):
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x), dim=1)
        return x

In [3]:
# model sanity_check
sample_net = VanillaFFN(784, 10)
print(sample_net(torch.randn(64, 784)).shape)

torch.Size([64, 10])


In [4]:
# Download Dataset
train_data = datasets.FashionMNIST(root='./data', 
                                   train=True, 
                                   transform=transforms.Compose([
                                       transforms.ToTensor()
                                   ]),
                                   download=True)
# Set up the data loader for our training data.
train_data_loader = DataLoader(dataset=train_data, 
                               shuffle=True,
                               batch_size=512)

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
Processing...
Done!


In [5]:
# sanity check for train data loader.
for idx, data_tuple in enumerate(train_data_loader):
    print(data_tuple[0].shape, data_tuple[1].shape)
    break


torch.Size([512, 1, 28, 28]) torch.Size([512])


In [6]:
# check and setup device tensor computations should be assigned to
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
INPUT_SIZE = 28 * 28
NUM_CLASSES = 10

vanilla_fnn_object = VanillaFFN(INPUT_SIZE, NUM_CLASSES)
vanilla_fnn_object.to(device=device)
loss_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vanilla_fnn_object.parameters(), lr=0.1)

for epoch in range(2):
    loss = 0
    for iteration, (data, targets) in enumerate(train_data_loader):
        # assign data and targets to device.
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # forward pass, compute losses, backpropagate.
        outputs = vanilla_fnn_object(data)
        loss = loss_criterion(outputs, targets)
        print("Epoch {}:, Iteration: {}, Loss: {}".format(epoch + 1, iteration + 1, loss))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
print("Learning Done!")

Epoch 1:, Iteration: 1, Loss: 2.3008344173431396
Epoch 1:, Iteration: 2, Loss: 40.24032211303711
Epoch 1:, Iteration: 3, Loss: 199.291259765625
Epoch 1:, Iteration: 4, Loss: 253.36212158203125
Epoch 1:, Iteration: 5, Loss: 148.43064880371094
Epoch 1:, Iteration: 6, Loss: 73.64132690429688
Epoch 1:, Iteration: 7, Loss: 21.558216094970703
Epoch 1:, Iteration: 8, Loss: 15.872228622436523
Epoch 1:, Iteration: 9, Loss: 7.039508819580078
Epoch 1:, Iteration: 10, Loss: 3.765284776687622
Epoch 1:, Iteration: 11, Loss: 3.8672595024108887
Epoch 1:, Iteration: 12, Loss: 3.136260747909546
Epoch 1:, Iteration: 13, Loss: 2.366891860961914
Epoch 1:, Iteration: 14, Loss: 2.2154788970947266
Epoch 1:, Iteration: 15, Loss: 2.349421262741089
Epoch 1:, Iteration: 16, Loss: 2.056389570236206
Epoch 1:, Iteration: 17, Loss: 1.727394938468933
Epoch 1:, Iteration: 18, Loss: 1.765850305557251
Epoch 1:, Iteration: 19, Loss: 1.9894050359725952
Epoch 1:, Iteration: 20, Loss: 1.6393438577651978
Epoch 1:, Iteration: 

In [8]:
# set up testing data and data loader
test_data = datasets.FashionMNIST(root='./data',
                                  train=False,
                                  download=True,
                                  transform=transforms.Compose([
                                      transforms.ToTensor()
                                  ]))

test_data_loader = DataLoader(dataset=test_data,
                              shuffle=True,
                              batch_size=512)

In [9]:
# Lets check the accuracy on the testing set.
num_correct = 0
num_samples = 0
vanilla_fnn_object.eval()    # to set in eval mode

with torch.no_grad():
    for iteration, (data, targets) in enumerate(test_data_loader):
        # assign data and targets to device.
        data = data.to(device=device)
        targets = targets.to(device=device)
        outputs = vanilla_fnn_object(data)
        
        values, index_of_max_value = outputs.max(1)
        num_correct += (index_of_max_value == targets).sum()
        num_samples += targets.size(0)
    
    print(num_correct, num_samples)
    print("Testing Accuracy of Vanilla FNN model on the FMNIST dataset: {}".format((num_correct.item() / num_samples) * 100))

tensor(7144, device='cuda:0') 10000
Testing Accuracy of Vanilla FNN model on the FMNIST dataset: 71.44
